In [ ]:
!pip install wikipedia unidecode

In [109]:
#CONFIG
max_letters = 12
language_tags = {

                'en':['actor', 'alcohol', 'cheque', 'cancer', 'chocolate', 'debate', 'hobby', 'melon', 'propaganda',
                      'religion', 'violin', 'england', 'MediaWiki','football','Napoleon','Money','Tiger','eagle','cricket','Finance','military','government','Emotion'],

                'cs': ['praha', 'evropa', 'pyreneje', 'voda', 'housle', 'Náboženství', 'Příroda', 'Ekosystém',
                    'vzdělání', 'Dům', 'Zpěvák', 'Zeus', 'Mykény', 'Starověké_Řecko', 'Renesance',
                    'Andrej_Babiš', 'Správa_železniční_dopravní_cesty', 'Kraje_v_Česku', 'Česko', 'Slezsko',
                    'Latina', 'Spojené_království', 'Římský_senát'],

                'de': ['Deutsche_Sprache', 'Deutschland', 'Kommunistische_Partei_der_Sowjetunion', 'Wasser',
                    'Festkörper', 'Seele', 'Geist', 'Dreifaltigkeit', 'Große', 'Christentum', 'Gott'],

                'sv': ['Svenska', 'Sverige', 'Danmark', 'Europeiska_unionen', 'Medeltiden', 'Feodalism', 'Kung',
                    'Kejsare', 'Monarki', 'Valmonarki', 'Choklad', 'Mjölk', 'Prolaktin', 'Kvinna', 'Eldvapen',
                    'Kina', 'Götar', 'Romantiken', 'Ideologi', 'Tänkande', 'Pedagogik', 'Sekund', 'Solen', 'Väder',
                    'Mellanöstern', 'Väte', 'Anatomi', 'Hjärta', 'Puls', 'Grekiska', 'Cypern'],

                'fr': ['Français', 'Langues_romanes', 'Charlemagne', 'Traité_de_Verdun', 'Louis_le_Pieux',
                    'Son_(physique)', 'Zoologie', 'Intelligence_animale', 'Intelligence', 'Tautologie',
                    'Pléonasme', 'Figure_de_style']

                 }

In [110]:
import re
import wikipedia as wiki
from unidecode import unidecode
import pandas as pd
import numpy as np

def process(page_content, max_word_length):
    words = re.sub(r'[^a-zA-Z ]', '', page_content)
    lower = words.lower()
    word_list = lower.split()
    short_words = []
    for word in word_list:
        if len(word) <= max_word_length:
            short_words.append(word)
    return short_words

def generate_dictionary(tag, max_word_length):

    wiki.set_lang(tag)
    for topic in language_tags[tag]:
        page = wiki.WikipediaPage(topic)
        content = page.content
        content = unidecode(content)
        lst = process(content, max_word_length)

    return lst

def convert_dic_to_vector(dic, max_word_length):
    new_list = []
    for word in dic:
        vec = ''
        n = len(word)
        for i in range(n):
            current_letter = word[i]
            ind = ord(current_letter)-97
            placeholder = (str(0)*ind) + str(1) + str(0)*(25-ind)
            vec = vec + placeholder
        if n < max_word_length:
            excess = max_word_length-n
            vec = vec +str(0)*26*excess
        new_list.append(vec)
    print(len(new_list))
    return new_list

def create_output_vector(tag_index, number_of_languages):
    out = str(0)*tag_index + str(1) + str(0)*(number_of_languages-1-tag_index)
    return out


In [ ]:
word_data = []
language_data = []
master_dic = []

count = 0

for tag in language_tags.keys():
    print('generating dictionary for ' + tag)
    dic = generate_dictionary(tag, max_letters)
    for word in dic:
        master_dic.append(word)
    vct = convert_dic_to_vector(dic, max_letters)
    for vector in vct:
        word_data.append(vector)
    output_vct = create_output_vector(count, len(language_tags))
    for i in range(len(vct)):
        language_data.append(output_vct)
    count += 1
arr = []
for i in range(len(word_data)):
    entry = []
    entry.append(master_dic[i])
    for digit in language_data[i]:
        entry.append(float(digit))
    for digit in word_data[i]:
        entry.append(float(digit))
    arr.append(entry)


arr = np.array(arr)
np.save('arr.npy', arr)
df=pd.DataFrame(arr)
df.to_csv('data.csv')

In [112]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.model_selection import train_test_split


data = arr

inputs = data[:, 1+len(language_tags):]
labels = data[:, 1:1+len(language_tags)]

x_train, x_test, y_train, y_test = train_test_split(inputs, labels, test_size=0.15)

print(x_test.shape)
print(y_test.shape)
print(x_train.shape)
print(y_train.shape)

(5681, 312)
(5681, 5)
(32190, 312)
(32190, 5)


In [113]:
import tensorflow as tf
x_train=tf.strings.to_number(x_train, out_type=tf.dtypes.float32, name=None)
y_test=tf.strings.to_number(y_test, out_type=tf.dtypes.float32, name=None)
x_test=tf.strings.to_number(x_test, out_type=tf.dtypes.float32, name=None)
y_train=tf.strings.to_number(y_train, out_type=tf.dtypes.float32, name=None)

In [114]:
network = Sequential()
network.add(Dense(200, input_dim=26*max_letters, activation='tanh'))
network.add(Dense(150, activation='tanh'))
network.add(Dense(100, activation='tanh'))
network.add(Dense(100, activation='tanh'))
network.add(Dense(len(language_tags), activation='softmax'))

network.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [115]:
!rm -rf ./logs/

In [ ]:
filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
tboard = TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
callbacks_list = [checkpoint, tboard]

network.fit(x_train, y_train, epochs=100, batch_size=1000, validation_data=(x_test, y_test), callbacks=callbacks_list)


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/

In [ ]:
network.load_weights('/content/weights.hdf5')
while True:
    dic = []
    valid = False
    while not valid:
        word = input('Enter word to predict:\n')
        if len(word) <= max_letters:
            word = word.lower()
            valid = True
        else:
            print('Word must be less than ' + str(max_letters + 1) + ' letters long')
    dic.append(word)
    vct_str = convert_dic_to_vector(dic, max_letters)
    vct = np.zeros((1, 26 * max_letters))
    count = 0
    for digit in vct_str[0]:
        vct[0,count] = int(digit)
        count += 1
    prediction_vct = network.predict(vct)

    langs = list(language_tags.keys())
    for i in range(len(language_tags)):
        lang = langs[i]
        score = prediction_vct[0][i]
        print(lang + ': ' + str(round(100*score, 2)) + '%')
    print('\n')
